In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import sent_tokenize,TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet,stopwords
from nltk import pos_tag
import texthero as hero
from num2words import num2words
import string
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import language_tool_python,random,sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
# file_1_path = './Data Files/ted_bundy_wiki_clean.txt'
# file_2_path = './Data Files/lotr1.txt'
path1 = './Data Files/harry_potter.pdf'
path2 = './Data Files/percy_jackson.pdf'

In [3]:
##################### READING TEXT FILES ###############################

def read_data_files(path1,path2,threshold=1000):
    file1 = open(path1,'r').read()
    file2 = open(path2,'r').read()
    
    file1 = sent_tokenize(file1)
    file2 = sent_tokenize(file2)
    return file1[:min(len(file1),threshold)],file2[:min(len(file2),threshold)]

In [4]:
###################### READING PDFS ####################################

import PyPDF2
from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

def return_output_string(path):
    output_string = StringIO()
    with open(path, 'rb') as in_file:
        parser = PDFParser(in_file)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.create_pages(doc):
            interpreter.process_page(page)
    
    return output_string.getvalue()

def read_pdfs(path1,path2,threshold=1000):
    file1 = return_output_string(path1)
    file2 = return_output_string(path2)
    
    file1 = sent_tokenize(file1)
    file2 = sent_tokenize(file2)
    
    return file1[5:min(len(file1),threshold)],file2[5:min(len(file2),threshold)]

In [5]:
#corpus1,corpus2 = read_data_files(file_1_path,file_2_path)
corpus1,corpus2 = read_pdfs(path1,path2)

In [6]:
print(len(corpus1),len(corpus2))

995 995


In [7]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
def basic_preprocessing(series):
    
    # Removing brackets
    series = hero.remove_brackets(series)
    
    # Removing diacritics
    series = hero.remove_diacritics(series) # Words like Café, that top extra char will be removed
    
    # Removing whitespaces
    series = hero.remove_whitespace(series)
    
    series = hero.remove_digits(series)
                      
    return series

def custom_preprocessing(text):
    text = re.sub(r"[^A-Za-z0-9,!.']", " ", text)
    text = re.sub(r"(\s)(\s)+",' ',text)
    return text


def list_to_string(sentence_list):
    text = ''
    for row in sentence_list:
        if text is '':
            text += row+'\n'
        else:
            text += ' ' + row+'\n'
    return text

def string_to_list(sentence_string):
    sentences = sent_tokenize(sentence_string)
    return sentences

In [9]:
corpus1 = basic_preprocessing(pd.Series(corpus1)).tolist()
corpus2 = basic_preprocessing(pd.Series(corpus2)).tolist()

corpus1 = pd.Series(corpus1).apply(custom_preprocessing).tolist()
corpus2 = pd.Series(corpus2).apply(custom_preprocessing).tolist()

In [10]:
max_words = 10000

embeddings_index = {}
f = open('glove.6B.50d.txt','r',encoding='utf-8')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float32')
    embeddings_index[word] = coefs

f.close()

In [11]:
tokenizer = TreebankWordTokenizer()

def find_most_similar(input_sentence_embedding, target_sentence_embeddings, ignore_list):
    similarity = [cosine_similarity([input_sentence_embedding], [target_sentence])[0][0] for target_sentence in target_sentence_embeddings]
    if len(ignore_list) > 0 :
        for i in ignore_list : 
            similarity[i] = -1
    most_similar_index = np.argmax(similarity)
    return most_similar_index, similarity[most_similar_index]

def prepare_freq_table(corpus1,corpus2):
    counter = Counter()
    corpus1 = list_to_string(corpus1)
    corpus2 = list_to_string(corpus2)
    tokens1 = [word for word in tokenizer.tokenize(corpus1)]
    tokens2 = [word for word in tokenizer.tokenize(corpus2)]
    
    counter.update(tokens1)
    counter.update(tokens2)
    return dict(counter)
    


def prepare_sentence_embeddings(corpus,index,freqs,dimensions=50,a=0.001):
    total_freq = sum(freqs.values())
    embeddings = []
    stopwords_list = stopwords.words('english')
    
    for sentence in corpus:
        sentence = sentence.lower()
        sentence = re.sub(r"[^A-Za-z]", " ", sentence)
        sentence = re.sub(r"(\s)(\s)+"," ",sentence)
        tokens = list(set([word for word in tokenizer.tokenize(sentence) if word not in stopwords_list and word in index.keys()]))
        weights = [a/(a+freqs.get(token,0)/total_freq) for token in tokens]
        if len(tokens) == 0:
            embeddings.append(np.zeros((dimensions,)))
        else:
            embedding = np.average([index[token] for token in tokens], axis=0, weights=weights)
            embeddings.append(embedding)
    
    return embeddings
    
    
def similarity_module(corpus1,corpus2,index):
    freq_dict = prepare_freq_table(corpus1,corpus2)
    corpus_1_embeddings = prepare_sentence_embeddings(corpus1,index,freq_dict)
    corpus_2_embeddings = prepare_sentence_embeddings(corpus2,index,freq_dict)
    similarity_df = pd.DataFrame(columns=['Source','Target','Similarity_Value'])
    ignore_list = []
    for i,e1 in enumerate(corpus_1_embeddings):
        index,value = find_most_similar(e1,corpus_2_embeddings,ignore_list)
        ignore_list.append(index)
        similarity_df.loc[i,'Source'] = i
        similarity_df.loc[i,'Target'] = index
        similarity_df.loc[i,'Similarity_Value'] = value
    
    return similarity_df
        
    #print(find_most_similar(corpus_1_embeddings[2],corpus_2_embeddings))
    #print(len(corpus1),len(corpus_1_embeddings))
    #print(len(corpus2),len(corpus_2_embeddings))

In [13]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import re
import heapq

def prepare_freq_table(corpus1,corpus2):
    counter = Counter()
    corpus1 = list_to_string(corpus1)
    corpus2 = list_to_string(corpus2)
    tokens1 = [word for word in tokenizer.tokenize(corpus1)]
    tokens2 = [word for word in tokenizer.tokenize(corpus2)]
    
    counter.update(tokens1)
    counter.update(tokens2)
    return dict(counter)

def NER_tags_creator(sentence,model):
    ner_results = model(sentence)
    token = None
    tag = None
    ner_pairs = []
    i = 0

    while i < len(ner_results):
        if tag is None:
            token = ner_results[i]['word']
            tag = ner_results[i]['entity'][2:]

        elif 'B' in ner_results[i]['entity']:
            ner_pairs.append((tag,token))
            token = ner_results[i]['word']
            tag = ner_results[i]['entity'][2:]

        else:
            token += ' '+ner_results[i]['word']   
            token = re.sub(r'[#]+','',token)
        i+= 1

    ner_pairs.append((tag,token))
    ner_pairs = list(set(ner_pairs))
    return ner_pairs

def build_list_for_heap(pairs,freq_table):
    person_heap = []
    org_loc_heap = []
    
    for tag,word in pairs:
        freq = -1*freq_table.get(word,0)
        if tag == "PER":
            person_heap.append((freq,word))
        
        elif tag in ['LOC','ORG']:
            org_loc_heap.append((freq,word))
    
    return person_heap,org_loc_heap

def generate_swappers(s1,s2,model,freq_table): # Main function (CALL THIS FUNCTION) with two sentences s1,s2 which are similar(similarity module)
    
    s1_ner_pairs = NER_tags_creator(s1,model)
    s2_ner_pairs = NER_tags_creator(s2,model)
    
    person_heap_s1, org_loc_heap_s1 = build_list_for_heap(s1_ner_pairs,freq_table)
    person_heap_s2, org_loc_heap_s2 = build_list_for_heap(s2_ner_pairs,freq_table)
    
    heapq.heapify(person_heap_s1)
    heapq.heapify(person_heap_s2)
    
    heapq.heapify(org_loc_heap_s1)
    heapq.heapify(org_loc_heap_s2)
    
    new_sentences = []
    
    if len(person_heap_s1) != 0 and len(person_heap_s2) != 0:
        word_s1 = heapq.heappop(person_heap_s1)[1]
        word_s2 = heapq.heappop(person_heap_s2)[1]
        
        new_sentences.append(s1.replace(word_s1,word_s2))
        new_sentences.append(s2.replace(word_s2,word_s1))
    
    
    if len(org_loc_heap_s1) != 0 and len(org_loc_heap_s2) != 0:
        word_s1 = heapq.heappop(org_loc_heap_s1)[1]
        word_s2 = heapq.heappop(org_loc_heap_s2)[1]
        
        new_sentences.append(s1.replace(word_s1,word_s2))
        new_sentences.append(s2.replace(word_s2,word_s1))
    
    return new_sentences

In [14]:
def create_data_plus_ner_swap(corpus1,corpus2,sm_df,source,target,value_name,model,threshold=0.7):
    freq_mapper =  prepare_freq_table(corpus1,corpus2)
    sm_df = sm_df[sm_df[value_name] > threshold].sort_values(by=source)
    final_data = []
    ner_samples = []
    indicator = []
    for i,j in zip(sm_df[source],sm_df[target]):
        s1 = corpus1[i]
        s2 = corpus2[j]
        samples = generate_swappers(s1,s2,nlp,freq_mapper)
        if len(samples) > 0:
            indicator.append((i,j))
        final_data.append(s1) # S1 sentence
        final_data.append(s2) # S2 sentence
        final_data.extend(samples) # NER Swaps added
        ner_samples.extend(samples) # All NER swaps samples
    
    ner_samples = [s for s in ner_samples if s not in corpus1 and s not in corpus2]
    return final_data,ner_samples,indicator

In [15]:
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)

In [16]:
ss_df = similarity_module(corpus1,corpus2,embeddings_index)

Token indices sequence length is longer than the specified maximum sequence length for this model (17522 > 512). Running this sequence through the model will result in indexing errors


In [17]:
final_data,ner_samples,indicator = create_data_plus_ner_swap(corpus1,corpus2,ss_df,'Source','Target','Similarity_Value',nlp)

In [18]:
indicator[:10]

[(0, 474),
 (1, 58),
 (2, 634),
 (5, 90),
 (7, 762),
 (8, 167),
 (11, 17),
 (15, 339),
 (21, 232),
 (22, 707)]

In [19]:
final_data[:5]

['The Dursleys had a small son called Dudley and in their opinion there was no finer boy anywhere.',
 "something like The card was in fancy script, which was murder on my dyslexic eyes, but I finally made out Grover Underwood Keeper Half Blood Hill Long Island, New York 009 0009 What's Half Don't say it aloud! ",
 'The Groversleys had a small son called Grovedley and in their opinion there was no finer boy anywhere.',
 "something like The card was in fancy script, which was murder on my dyslexic eyes, but I finally made out Dur Underwood Keeper Half Blood Hill Long Island, New York 009 0009 What's Half Don't say it aloud! ",
 'The Dursleys had everything they wanted, but they also had a secret, and their greatest fear was that somebody would discover it.']

In [20]:
ner_samples[:5]

['The Groversleys had a small son called Grovedley and in their opinion there was no finer boy anywhere.',
 "something like The card was in fancy script, which was murder on my dyslexic eyes, but I finally made out Dur Underwood Keeper Half Blood Hill Long Island, New York 009 0009 What's Half Don't say it aloud! ",
 'The Doddrsleys had everything they wanted, but they also had a secret, and their greatest fear was that somebody would discover it.',
 'I was trying to listen to what he had to say, because it was kind of interesting, but everybody around me was talking, and every time I told them to shut up, the other teacher chaperone, Mrs. Dus, would give me the evil eye.',
 "They didn't think they could bear it if anyone found out about the Gabes."]

In [23]:
import joblib

joblib.dump(final_data,'final_data')

['final_data']

In [26]:
joblib.__version__

'0.17.0'

In [25]:
x = joblib.load('final_data')
type(x)

list

In [21]:
def create_training_data(data,maxlen=180,step=3):
    data = list_to_string(data)
    data = data.lower()
    data = re.sub(r"[^A-Za-z.']", " ", data)
    data = re.sub(r"(\s)(\s)+"," ",data)
    
    sentences = []
    next_chars = []
    for i in range(0,len(data)-maxlen,step):
        sentences.append(data[i:i+maxlen])
        next_chars.append(data[i+maxlen])
    
    print(f'The no of rows will be {len(sentences)}')
    
    chars = sorted(list(set(data))) # Consist of all unique characters in the file
    char_indices = dict((char,chars.index(char)) for char in chars) # character to index mapping
    print(f'The length of unique characters are {len(chars)}')
    print(chars)
    
    x = np.zeros((len(sentences),maxlen,len(chars)))
    y = np.zeros((len(sentences),len(chars)))
    
    for i,sentence in enumerate(sentences):
        for t,char in enumerate(sentence):
            x[i,t,char_indices[char]] = 1
        y[i,char_indices[next_chars[i]]] = 1
    return x,y,len(chars),chars,char_indices,sentences

In [22]:
maxlen=180
X,y,len_chars,chars,char_indices,sentences = create_training_data(final_data)

The no of rows will be 45173
The length of unique characters are 29
[' ', "'", '.', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


# MODEL BUILDING

In [23]:
from keras.models import Sequential,load_model
from keras.layers import LSTM,Dense,Dropout,GRU
from keras.optimizers import Adam

In [24]:
model = Sequential()

model.add(GRU(256,input_shape=(maxlen,len_chars),recurrent_dropout=0.5))
model.add(Dropout(0.25))
model.add(Dense(len_chars,activation='softmax'))

model.compile(optimizer='rmsprop',loss='categorical_crossentropy')

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 256)               220416    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 29)                7453      
Total params: 227,869
Trainable params: 227,869
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.fit(X,y,batch_size=128,epochs=100,validation_split=0.2,shuffle=False)

Epoch 1/100
283/283 [==============================] - 218s 765ms/step - loss: 2.8473 - val_loss: 2.3597
Epoch 2/100
283/283 [==============================] - 218s 771ms/step - loss: 2.2989 - val_loss: 2.2203
Epoch 3/100
283/283 [==============================] - 227s 801ms/step - loss: 2.1912 - val_loss: 2.1488
Epoch 4/100
283/283 [==============================] - 235s 830ms/step - loss: 2.1243 - val_loss: 2.0905
Epoch 5/100
283/283 [==============================] - 237s 839ms/step - loss: 2.0699 - val_loss: 2.0400
Epoch 6/100
283/283 [==============================] - 229s 811ms/step - loss: 2.0245 - val_loss: 1.9975
Epoch 7/100
283/283 [==============================] - 230s 815ms/step - loss: 1.9745 - val_loss: 1.9659
Epoch 8/100
283/283 [==============================] - 229s 808ms/step - loss: 1.9426 - val_loss: 1.9321
Epoch 9/100
283/283 [==============================] - 229s 809ms/step - loss: 1.9088 - val_loss: 1.9093
Epoch 10/100
283/283 [==============================] -

In [27]:
# model.save('model_crossover_50_epochs_shuffle_false.h5')

#model.save('model_crossover_100_gru_256_harry_percy.h5')

#model = load_model('model_crossover_50_gru_256_harry_percy.h5')

In [28]:
def sample(preds,temperature=0.4):
    preds = np.asarray(preds).astype('float64')
    
    preds = np.log(preds)/temperature # Here temperature acts like a amplifying factor. Lower value will amplify the
                                      # most probable character and it will come again and again
    exp_preds = np.exp(preds)
    
    exp_preds = exp_preds/np.sum(exp_preds)
    
    probas = np.random.multinomial(1,exp_preds,1)
    
    return np.argmax(probas)
    

In [29]:
import tensorflow as tf
from tensorflow.nn import softmax

# def topK(preds,k=5):
#     preds = np.asarray(preds).astype('float64')

#     top_k_probs, top_k_indices = tf.math.top_k(preds, k=k, sorted=True)
#     # will return an sorted array containing the probs of the top-k elements & the indices
    
#     top_k_probs_redistributed = np.asarray(softmax(top_k_probs)).astype("float")
    
#     sampled_token = np.random.choice(top_k_indices, p=top_k_probs_redistributed)

#     return sampled_token

def topK(predictions, k=5):
    top_k_probabilities, top_k_indices= tf.math.top_k(predictions, k=k, sorted=True)
    top_k_indices = np.asarray(top_k_indices).astype("int32")
    top_k_redistributed_probability=softmax(np.log(top_k_probabilities))
    top_k_redistributed_probability = np.asarray(top_k_redistributed_probability).astype("float32")
    sampled_token = np.random.choice(top_k_indices, p=top_k_redistributed_probability)
    return sampled_token


def topP(preds,p=.9):
    preds = np.asarray(preds).astype('float64')
    preds1 = np.copy(preds) 
    x,y=[],[] # x will contain indices, y will have values
    small=True 
    while small==True:
        p1 = np.argmax(preds1) # take index of highest value
        x.append(p1) # append index of highest value
        y.append(preds1[p1]) # append value of highest value
        if sum(y) > p: 
            x=x[:-1] # if sum became greater than the threshold, just remoce the last values of x & y
            y=y[:-1]
            small=False
        preds1[p1]=-1000000000000000000000000000000000
    if len(x)>0 : 
        top_k_probs_redistributed = np.asarray(softmax(y)).astype("float")
    
        sampled_token = np.random.choice(x, p=top_k_probs_redistributed)    
        return sampled_token
    else :
        return np.argmax(preds)

## DYNAMIC TEMPERATURE SAMPLING

In [30]:
np.random.seed(42)

#generate = "He was provoked Professor Snape said Hagrid sticking his huge hairy face out from behind the tree."
generate = "Zeus did indeed feed Kronos a mixture of mustard and wine which made him disgorge his other five children"
generate = generate + ' '*(maxlen - len(generate)) # To make it as the same length
generate = generate.lower()

result = generate

for i in range(2000):
    sampled = np.zeros((1,maxlen,len_chars))
    
    for t,char in enumerate(generate):
        sampled[0,t,char_indices[char]] = 1
    
    preds = model.predict(sampled)[0] 
    temperature = np.random.normal(0.5,0.1)
    next_char = chars[sample(preds,temperature)]
    generate += next_char
    result += next_char
    generate = generate[1:] # sliding window type. So you slide one step to the right and repeat
    

tool = language_tool_python.LanguageTool('en-US')
result_clean = tool.correct(result)

print(result_clean)

Zeus did indeed feed kronor a mixture of mustard and wine which made him disgorge his other five children the books of the screaming at the only came to say band his had been his had a got on the first the said the fruit stand and started no look the saw and on shoot something a motorcycle Vernon and the little school Nancy benefit made me was all over the started to say in a very looked and a little was tell miss or Gabe a silly under. I wanted about the Nancy bob for he didn't be said band and strutting his rears instead he was to the end the book to he looked all over the started the cat the bark, and he wasn't be a street secs the said of the name in he couldn't be the suppose so Mrs. Dodds was the cat had been the sign they couldn't be a friend of the front start Dudley was the notice he was notes band and was when he had a landed to the on the stare the sign the strange and the bout to say, but it was before I had talking out of she said professor McGonagall got me the light the 

## TOP K SAMPLING (k=7) 

In [31]:
np.random.seed(42)

#generate = "He was provoked Professor Snape said Hagrid sticking his huge hairy face out from behind the tree."
generate = "Zeus did indeed feed Kronos a mixture of mustard and wine which made him disgorge his other five children"
generate = generate + ' '*(maxlen - len(generate)) # To make it as the same length
generate = generate.lower()

result = generate

for i in range(2000):
    sampled = np.zeros((1,maxlen,len_chars))
    
    for t,char in enumerate(generate):
        sampled[0,t,char_indices[char]] = 1
    
    preds = model.predict(sampled)[0] 
    next_char = chars[topK(preds,7)]
    generate += next_char
    result += next_char
    generate = generate[1:] # sliding window type. So you slide one step to the right and repeat
    

tool = language_tool_python.LanguageTool('en-US')
result_clean = tool.correct(result)

print(result_clean)

Zeus did indeed feed kronor a mixture of mustard and wine which made him disgorge his other five children about the small and a letter a stow his making of a ho went his one wrong. At the books his none of the Dursley a Food it was had a bit he'll be seen so scraps we tell him but the small. He sits the looked a little was her in I have a women range. Dudley of that all her cup she looked in the sudden the door there was said. I was the close a bold and the could gen the sign to tell the Food light. They made ted it was had being of the only of the said then a cool when he wanted to bead him. Nake main body would sea mast brand. The pitching a big said her up wet uncle Vernon seen he was the said part on the bad of the cat something and stared back on the call to there were the bus and tell the attended her like he looked it the like and get the cure cat had and storms the looked out. This Lady of the book to be a mound which really and she Dudley would he potter went talking old datin

## TOP P SAMPLING (p = 0.9)

In [32]:
np.random.seed(42)

#generate = "He was provoked Professor Snape said Hagrid sticking his huge hairy face out from behind the tree."
generate = "Zeus did indeed feed Kronos a mixture of mustard and wine which made him disgorge his other five children"
generate = generate + ' '*(maxlen - len(generate)) # To make it as the same length
generate = generate.lower()

result = generate

for i in range(2000):
    sampled = np.zeros((1,maxlen,len_chars))
    
    for t,char in enumerate(generate):
        sampled[0,t,char_indices[char]] = 1
    
    preds = model.predict(sampled)[0] 
    next_char = chars[topP(preds)]
    generate += next_char
    result += next_char
    generate = generate[1:] # sliding window type. So you slide one step to the right and repeat
    

tool = language_tool_python.LanguageTool('en-US')
result_clean = tool.correct(result)

print(result_clean)

Zeus did indeed feed kronor a mixture of mustard and wine which made him disgorge his other five children he as though its fave it I couldn't long but the but sibling all meters. Feal you want of the math about a segment it looked legal Gabe band't he was poached SNEP which he didn't her out was snake look was for old day on the Olympia. They reader moves. For the school do books up the lad, but you looked to know Mr. funner here peeped by fame ray. Selestars in Mrs. Dudley married to Dudley boy for the sown in came until smell he couldn't too longs pig him in all punter had in one a class. Wer moss if as every snake back world being spent ion Percy just from the it in Mr. Dursley was now mother non his scratch could standing ALD coming as walking which school you could be over seller began. We are outside me for mastermind new indie and the boy area on the bass prove Mrs. Dodds from his pickers Dudley odds that bought sick friend be the fairly it met ricing he mane of anything me and 